In [1]:


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage,HumanMessage
import os


srt_dir = "../srt_files"
srt_array = [os.path.join(srt_dir, f) for f in os.listdir(srt_dir) if f.endswith(".srt")]


def return_srt_as_list_of_strings(srt_file:str)->list[str]:
    
    with open(srt_file, "r", encoding="utf-8") as f:
            srt_text = f.read()
            srt_list_file=srt_text.strip().split("\n\n") #Returns list 
    return srt_list_file
            
        
languages = [
    "English",
    "Mandarin Chinese",
    "Spanish",
    # "Hindi",
    # "Arabic",
    # "French",
    # "Bengali",
    # "Russian",
    # "Portuguese",
    # "Urdu",
    # "German",
    # "Japanese",
    # "Swahili",
    # "Turkish",
    # "Korean"
]
async def translate(chunkcs:str,language:str="English")-> str:
    llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.0)
    sys=SystemMessage("You are my translator of christianity srt files.You only returning transalted srt file with \n indicating new line.,dont include anythine lse only pure timestrempts with transaltion return")
    human=HumanMessage(content=f"Please transalte this in {language} here is srt file chunck: {chunkcs} ")
    results=await llm.ainvoke([sys,human])
    return results.content


def create_file(srt_path: str, srt_file: str, language: str):
    base = os.path.splitext(os.path.basename(srt_path))[0]
    filename = f"{base}_{language.replace(' ', '_')}.srt"
    output_path = os.path.join("../srt_files_translated", filename)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(srt_file)
    
    

async def transalte_chuncks():
    num_chunks = 10
    
    for srt in srt_array:
        srt_as_array=return_srt_as_list_of_strings(srt)
        total = len(srt_as_array)
        step = max(1, total // num_chunks)
        for lang in languages:
            translated_srt=[]
            for i in range(0,len(srt_as_array),step):
                chuncks=srt_as_array[i:i+step]
                chunks_to_translate="\n".join(chuncks) # pretvorimo tu listu od 100 elemenata u jedan tring 
                translated_chunck=await translate(chunks_to_translate,lang)
                translated_srt.append(translated_chunck)
            translated_srt_as_str= "\n".join(translated_srt).encode('utf-8').decode('unicode_escape')
            create_file(srt,translated_srt_as_str,lang)

    

    
    
    

In [2]:
await transalte_chuncks()

None
